In [ ]:
!pip install torch-summary

In [ ]:
import pandas as pd
from google.colab import drive
import gspread

# Mounting Google Drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

df_test = pd.read_excel("/content/drive/MyDrive/Elorm's Work/test_dataframes.xlsx")
df_train = pd.read_excel("/content/drive/MyDrive/Elorm's Work/train_dataframes.xlsx")

In [ ]:
df_train

In [ ]:
# USING A NORMAL NEURAL NETWORK FOR SHORT-TERM LOAD FORECASTING

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from datetime import datetime

torch.manual_seed(0)
np.random.seed(0)
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

df_train['datetime'] = pd.to_datetime(df_train['datetime'])
df_test['datetime'] = pd.to_datetime(df_test['datetime'])

df_train['year'] = df_train['datetime'].dt.year
df_train['month'] = df_train['datetime'].dt.month
df_train['day'] = df_train['datetime'].dt.day
df_train['hour'] = df_train['datetime'].dt.hour
df_test['year'] = df_test['datetime'].dt.year
df_test['month'] = df_test['datetime'].dt.month
df_test['day'] = df_test['datetime'].dt.day
df_test['hour'] = df_test['datetime'].dt.hour

df_train.drop(columns=['datetime'], inplace=True)
df_test.drop(columns=['datetime'], inplace=True)

X_train = df_train.drop(columns=['DEMAND']).values
y_train = df_train['DEMAND'].values
X_test = df_test.drop(columns=['DEMAND']).values
y_test = df_test['DEMAND'].values

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

class EnergyDemandDataset(torch.utils.data.Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

# create datasets and dataloaders
train_dataset = EnergyDemandDataset(X_train, y_train)
test_dataset = EnergyDemandDataset(X_test, y_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

class ANNModel(nn.Module):
    def __init__(self, input_dim):
        super(ANNModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x

input_dim = X_train.shape[1]
model = ANNModel(input_dim)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 350

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}") # if you want to suppress the results, comment this out

def evaluate_model(loader, model):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for inputs, targets in loader:
            outputs = model(inputs)
            predictions.extend(outputs.squeeze().tolist())
            actuals.extend(targets.tolist())
    return np.array(predictions), np.array(actuals)

# evaluate the model with the actual test data
predictions, actuals = evaluate_model(test_loader, model)

mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actuals, predictions)
mape = np.mean(np.abs((actuals - predictions) / actuals)) * 100

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"MAPE: {mape}%")

In [ ]:
# USING A CONVOLUTIONAL NEURAL NETWORK FOR SHORT-TERM LOAD FORECASTING

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

torch.manual_seed(0)
np.random.seed(0)
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

X_train = df_train.drop(columns=['DEMAND']).values
y_train = df_train['DEMAND'].values
X_test = df_test.drop(columns=['DEMAND']).values
y_test = df_test['DEMAND'].values

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

class EnergyDemandDataset(torch.utils.data.Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

train_dataset = EnergyDemandDataset(X_train, y_train)
test_dataset = EnergyDemandDataset(X_test, y_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

class ComplexCNNModel(nn.Module):
    def __init__(self):
        super(ComplexCNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.5)

        self._to_linear = None
        self.convs = nn.Sequential(
            self.conv1,
            nn.ReLU(),
            self.pool,
            self.conv2,
            nn.ReLU(),
            self.pool,
            self.conv3,
            nn.ReLU(),
            self.pool
        )
        x = torch.randn(64, 1, X_train.shape[2]).float()  # batch size, channels, features
        self._to_linear = self.convs(x).view(x.size(0), -1).shape[1]

        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.convs(x)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x

model = ComplexCNNModel()

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 350

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

def evaluate_model(loader, model):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for inputs, targets in loader:
            outputs = model(inputs)
            predictions.extend(outputs.squeeze().tolist())
            actuals.extend(targets.tolist())
    return np.array(predictions), np.array(actuals)

predictions, actuals = evaluate_model(test_loader, model)

mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actuals, predictions)
mape = np.mean(np.abs((actuals - predictions) / actuals)) * 100

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"MAPE: {mape}%")


In [ ]:
# USING A RECURRENT NEURAL NETWORK FOR SHORT-TERM LOAD FORECASTING

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

torch.manual_seed(0)
np.random.seed(0)
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

X_train = df_train.drop(columns=['DEMAND']).values
y_train = df_train['DEMAND'].values
X_test = df_test.drop(columns=['DEMAND']).values
y_test = df_test['DEMAND'].values

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

seq_length = 1
X_train = X_train.reshape(X_train.shape[0], seq_length, -1)
X_test = X_test.reshape(X_test.shape[0], seq_length, -1)

class EnergyDemandDataset(torch.utils.data.Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

train_dataset = EnergyDemandDataset(X_train, y_train)
test_dataset = EnergyDemandDataset(X_test, y_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(RNNModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc1(out[:, -1, :])
        out = torch.relu(out)
        out = self.fc2(out)
        out = torch.relu(out)
        out = self.fc3(out)
        return out

input_dim = X_train.shape[2]
hidden_dim = 128
num_layers = 3

model = RNNModel(input_dim, hidden_dim, num_layers)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 350

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

def evaluate_model(loader, model):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for inputs, targets in loader:
            outputs = model(inputs)
            predictions.extend(outputs.squeeze().tolist())
            actuals.extend(targets.tolist())
    return np.array(predictions), np.array(actuals)

predictions, actuals = evaluate_model(test_loader, model)

mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actuals, predictions)
mape = np.mean(np.abs((actuals - predictions) / actuals)) * 100

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"MAPE: {mape}%")


In [ ]:
# USING A LONG SHORT-TERM NEURAL NETWORK MODEL FOR SHORT-TERM LOAD FORECASTING

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

torch.manual_seed(0)
np.random.seed(0)
X_train = df_train.drop(columns=['DEMAND']).values
y_train = df_train['DEMAND'].values
X_test = df_test.drop(columns=['DEMAND']).values
y_test = df_test['DEMAND'].values

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

seq_length = 1
X_train = X_train.reshape(X_train.shape[0], seq_length, -1)
X_test = X_test.reshape(X_test.shape[0], seq_length, -1)

class EnergyDemandDataset(torch.utils.data.Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

train_dataset = EnergyDemandDataset(X_train, y_train)
test_dataset = EnergyDemandDataset(X_test, y_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

class ComplexLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout_prob):
        super(ComplexLSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout_prob)
        self.fc1 = nn.Linear(hidden_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])  # added a dropout for shege reasons. but keep it here. it breaks without it for some reason.
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        out = self.relu(self.fc2(out))
        out = self.dropout(out)
        out = self.relu(self.fc3(out))
        out = self.fc4(out)
        return out

input_dim = X_train.shape[2]
hidden_dim = 256
num_layers = 3
dropout_prob = 0.3

model = ComplexLSTMModel(input_dim, hidden_dim, num_layers, dropout_prob)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 350

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

def evaluate_model(loader, model):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for inputs, targets in loader:
            outputs = model(inputs)
            predictions.extend(outputs.squeeze().tolist())
            actuals.extend(targets.tolist())
    return np.array(predictions), np.array(actuals)

predictions, actuals = evaluate_model(test_loader, model)

mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actuals, predictions)
mape = np.mean(np.abs((actuals - predictions) / actuals)) * 100

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"MAPE: {mape}%")

In [ ]:
# USING A CNN-LSTM MODEL FOR SHORT-TERM LOAD FORECASTING

from torchsummary import summary
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

torch.manual_seed(0)
np.random.seed(0)

X_train = df_train.drop(columns=['DEMAND']).values
y_train = df_train['DEMAND'].values
X_test = df_test.drop(columns=['DEMAND']).values
y_test = df_test['DEMAND'].values

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

seq_length = 1
X_train = X_train.reshape(X_train.shape[0], seq_length, -1)
X_test = X_test.reshape(X_test.shape[0], seq_length, -1)

class EnergyDemandDataset(torch.utils.data.Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

train_dataset = EnergyDemandDataset(X_train, y_train)
test_dataset = EnergyDemandDataset(X_test, y_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

class HybridCNNLSTMModel(nn.Module):
    def __init__(self, input_dim, cnn_out_channels, lstm_hidden_dim, lstm_num_layers, fc_hidden_dim, dropout_prob):
        super(HybridCNNLSTMModel, self).__init__()
        self.cnn_out_channels = cnn_out_channels
        self.lstm_hidden_dim = lstm_hidden_dim
        self.lstm_num_layers = lstm_num_layers

        self.conv1 = nn.Conv1d(in_channels=1, out_channels=cnn_out_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=cnn_out_channels, out_channels=cnn_out_channels, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)

        cnn_output_dim = (input_dim + 1) // 2 * cnn_out_channels

        self.lstm = nn.LSTM(cnn_output_dim, lstm_hidden_dim, lstm_num_layers, batch_first=True, dropout=dropout_prob)

        self.fc1 = nn.Linear(lstm_hidden_dim, fc_hidden_dim)
        self.fc2 = nn.Linear(fc_hidden_dim, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        batch_size, seq_len, input_dim = x.shape
        x = x.view(batch_size * seq_len, 1, input_dim)

        x = self.relu(self.conv1(x))
        x = self.pool(self.relu(self.conv2(x)))

        cnn_out_dim = x.shape[2]
        x = x.view(batch_size, seq_len, cnn_out_dim * self.cnn_out_channels)

        h0 = torch.zeros(self.lstm_num_layers, batch_size, self.lstm_hidden_dim).to(x.device)
        c0 = torch.zeros(self.lstm_num_layers, batch_size, self.lstm_hidden_dim).to(x.device)
        out, _ = self.lstm(x, (h0, c0))

        out = self.dropout(out[:, -1, :])
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        out = self.relu(self.fc2(out))
        out = self.fc3(out)
        return out

input_dim = X_train.shape[2]
cnn_out_channels = 64
lstm_hidden_dim = 128
lstm_num_layers = 2
fc_hidden_dim = 256
dropout_prob = 0.3

model = HybridCNNLSTMModel(input_dim, cnn_out_channels, lstm_hidden_dim, lstm_num_layers, fc_hidden_dim, dropout_prob)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 3

summary(model)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

def evaluate_model(loader, model):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for inputs, targets in loader:
            outputs = model(inputs)
            predictions.extend(outputs.squeeze().tolist())
            actuals.extend(targets.tolist())
    return np.array(predictions), np.array(actuals)

predictions, actuals = evaluate_model(test_loader, model)

mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actuals, predictions)
mape = np.mean(np.abs((actuals - predictions) / actuals)) * 100

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"MAPE: {mape}%")